In [146]:
import pandas as pd
PATH = "~/data/movies/dailyboxoffice/dt=20240101"
df = pd.read_parquet(PATH)
len(df)

50

In [147]:
df.groupby(['multiMovieYn', 'repNationCd'], dropna=False, observed=False) \
.size() \
.reset_index(name='count')

,multiMovieYn,repNationCd,count
0,N,F,0
1,N,K,0
2,N,NaN,10
3,Y,F,0
4,Y,K,0
5,Y,NaN,10
6,NaN,F,10
7,NaN,K,10
8,NaN,NaN,10


In [148]:
for i, row in df.iterrows():
    if i > 2:
        break
    if pd.isna(row['multiMovieYn']):
        print(f"{row['movieCd']}")

20203702
20212866
20236146


In [149]:
r = df[df['movieCd'] == '20203702'][['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]
print(r.to_markdown())

|    |   movieCd | movieNm           | multiMovieYn   | repNationCd   |
|---:|----------:|:------------------|:---------------|:--------------|
|  0 |  20203702 | 노량: 죽음의 바다 | nan            | nan           |
| 10 |  20203702 | 노량: 죽음의 바다 | N              | nan           |
| 40 |  20203702 | 노량: 죽음의 바다 | nan            | K             |


In [150]:
def fill_na_with_column(origin_df, c_name):
    df = origin_df.copy()
    for i, row in df.iterrows():
        if pd.isna(row[c_name]):
            same_movie_df = df[df["movieCd"] == row["movieCd"]]
            notna_idx = same_movie_df[c_name].dropna().first_valid_index()
            if notna_idx is not None:
                df.at[i, c_name] = df.at[notna_idx, c_name]
    return df

In [151]:
c_name = 'multiMovieYn'
filed_df_1 = fill_na_with_column(df, c_name)
print("원천 NaN count:", df[c_name].isna().sum())
print("원천 NaN count:", filed_df_1[c_name].isna().sum())

원천 NaN count: 30
원천 NaN count: 5


In [152]:
c_name = 'repNationCd'
filed_df_2 = fill_na_with_column(filed_df_1, c_name)
print("원천 NaN count:", filed_df_1[c_name].isna().sum())
print("원천 NaN count:", filed_df_2[c_name].isna().sum())

원천 NaN count: 30
원천 NaN count: 5


In [154]:
len(filed_df_2)

50

In [156]:
# data frame 에서 multiMovieYn, repNationCd 두 컬럼 중 하나라도 NaN 인 것 ?
len(filed_df_2[filed_df_2[['multiMovieYn', 'repNationCd']].isna().any(axis=1)])

10

In [162]:
df_drop = filed_df_2.drop(columns=['rnum', 'rank', 'rankInten', 'salesShare'])

In [163]:
unique_df = df_drop.drop_duplicates()
len(unique_df)

25

In [165]:
unique_df.sort_values(by="movieNm")

,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd
18,OLD,20190324,3일의 휴가,2023-12-06,28109697,-5429437,-16.2,4863180386,3295,-626,-16.0,517836,132,157,N,K
7,OLD,20234114,괴물,2023-11-29,92381680,-3768994,-3.9,3884333955,8710,-524,-5.7,396137,187,263,Y,F
49,OLD,20227525,교토에서 온 편지,2023-12-06,207000,-67800,-24.7,58288197,29,-19,-39.6,6717,3,3,NaN,K
26,OLD,20234664,그대들은 어떻게 살 것인가,2023-10-25,3371362,-142718,-4.1,19543638004,318,-29,-8.4,2009716,17,20,Y,NaN
19,OLD,20235098,나폴레옹,2023-12-06,4056567,-89539,-2.2,2246030517,540,-20,-3.6,225912,23,25,N,NaN
46,OLD,20228560,너와 나,2023-10-25,2625839,-3233861,-55.2,330224803,192,-337,-63.7,33811,8,8,NaN,K
0,OLD,20203702,노량: 죽음의 바다,2023-12-20,2893509165,-686119312,-19.2,36926118105,290717,-59184,-16.9,3728850,1706,5793,N,K
6,OLD,20235290,도티와 영원의 탑,2023-12-27,100024717,-13487835,-11.9,488186693,10704,-1268,-10.6,52400,357,419,N,K
22,OLD,20236080,류이치 사카모토: 오퍼스,2023-12-27,31686950,-15234118,-32.5,317887981,3001,-1659,-35.6,34196,200,270,Y,F
25,OLD,20235693,리빙: 어떤 인생,2023-12-13,6942434,-1569966,-18.4,174717087,729,-181,-19.9,18551,20,23,Y,F


In [169]:
unique_df["rnum"] = unique_df["audiCnt"].rank(method="dense", ascending=False).astype(int)
unique_df["rank"] = unique_df["audiCnt"].rank(method="min", ascending=False).astype(int)

In [170]:
unique_df

,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,multiMovieYn,repNationCd,rnum,rank
0,OLD,20203702,노량: 죽음의 바다,2023-12-20,2893509165,-686119312,-19.2,36926118105,290717,-59184,-16.9,3728850,1706,5793,N,K,1,1
1,OLD,20212866,서울의 봄,2023-11-22,2666429137,-649456909,-19.6,118016920547,262422,-64054,-19.6,12117201,1614,5444,N,K,2,2
2,OLD,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,565417320,-69888800,-11.0,6071745009,58239,-6751,-10.4,626508,834,1671,N,F,3,3
3,OLD,20235735,아쿠아맨과 로스트 킹덤,2023-12-20,507984981,-156026099,-23.5,7967584753,49368,-14686,-22.9,772354,832,1611,N,F,4,4
4,OLD,20235596,트롤: 밴드 투게더,2023-12-20,273538754,-16945147,-5.8,3383181955,28988,-1952,-6.3,365927,660,981,N,F,5,5
5,OLD,20234673,뽀로로 극장판 슈퍼스타 대모험,2023-12-13,113680587,2278496,2.0,3046919883,11586,190,1.7,332776,335,432,N,K,6,6
6,OLD,20235290,도티와 영원의 탑,2023-12-27,100024717,-13487835,-11.9,488186693,10704,-1268,-10.6,52400,357,419,N,K,7,7
7,OLD,20234114,괴물,2023-11-29,92381680,-3768994,-3.9,3884333955,8710,-524,-5.7,396137,187,263,Y,F,8,8
8,OLD,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",2023-12-27,53094530,-707720,-1.3,372974290,6471,67,1.0,45294,207,248,Y,F,9,9
9,OLD,20235980,말하고 싶은 비밀,2023-12-13,38025184,-2349427,-5.8,868379725,3765,-164,-4.2,89597,95,127,N,F,10,10
